<a href="https://colab.research.google.com/github/mabittar/Portfolio/blob/master/ML11_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Método de Ensemble

Por que usar apenas um modelo de machine learning, por apenas uma forma de fazer previsão? Imagine que você poderia aproveitar o melhor de cada mundo, usar os pontos fortes de cada estimador e até mesmo combiná-los.

Esta notebook é apenas uma introdução de um conceito conhecido como Métodos de Ensemble.


Me inspirei no [post](https://sigmoidal.ai/metodo-de-ensemble-vantagens-da-combinacao-de-diferentes-estimadores/) do [Marcelo Randolfo](https://www.linkedin.com/in/marcelo-randolfo/) para elaborar esse notebook, porém no post originial ele utiliza a base de dados da competição do [Kaggle](https://www.kaggle.com/c/titanic) sobre sobreviventes ao naufrágio do Titanic.

<center><img alt="Titanic x Python" width="30%" src="https://sigmoidal.ai/wp-content/uploads/2019/07/Data-Science-Investigando-o-naufr%C3%A1gio-do-Titanic-2-1024x576.jpg"></center>







## O dataset

Como desafio pessoal, preferi utilizar outro dataset no meu exemplo. Os dados aqui foi retirado da [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) e representa uma abordagem baseada em dados para *prever o sucesso do campanha de marketing* de um banco Português. 

<center><img alt="Telemarketing" width="40%" src="https://conteudo.movidesk.com/wp-content/uploads/2019/05/Blog-08-05-Telemarketing.jpg"></center>



A página com maiores informações sobre os dados pode ser acessada [aqui](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing).

O dataset é composto por 17 variáveis (colunas) e 45.211 entradas (linhas).



### Dicionário de Varáveis:

*Dados Pessoais:*

   1 - age (numerico)

   2 - job : categoria de trabalho (categórica) 

   3 - marital : estado civil (categórica)

   4 - education : escolaridade (categórica)

   5 - default: Inandimplente?  (binária)

   6 - balance: rendimentos anuais em euros (numérica) 

   7 - housing: tem empréstimo residencial? (binária)
   
   8 - loan: tem empréstimo pessoal? (binária)

*Relativos a campanha de marketing:*

   9 - contact: typo do contato (categórica)
   
  10 - day: o dia do mês que foi feito o útimo contato. (numérica)

  11 - month: o mês do último contato. (categórica: "jan", "feb", "mar", ..., "nov", "dec")

  12 - duration: duração do último contato, em segundos (numérica)

Outras Características:

  13 - campaign: quantidade de contatos feitos durante a campanha com o cliente.(numérica)

  14 - pdays: número de dias que se passaram desde o último contato. (numérica, -1 representa que o cliente ainda não foi contactado)
  
  15 - previous: número de contado feitos com o cliente. (numérica)

  16 - poutcome: classificação de campanhas realizadas anteriormente (categórica: "unknown","other","failure","success")

**Variável Alvo:**

  17 - y - O cliente assinou o termo? (binária: "yes","no")

In [ ]:
#primeiramente vamos baixar o dataset para a plataforma do GoogleColab
from google.colab import files
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip


--2020-08-31 20:47:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 579043 (565K) [application/x-httpd-php]
Saving to: ‘bank.zip’

bank.zip            100%[===================>] 565.47K  2.20MB/s    in 0.3s    

2020-08-31 20:47:35 (2.20 MB/s) - ‘bank.zip’ saved [579043/579043]



In [ ]:
#como o arquivo esta no formato .zip iremos descompactá-lo
!unzip \*.zip  && rm *.zip

Archive:  bank.zip
  inflating: bank-full.csv           
  inflating: bank-names.txt          
  inflating: bank.csv                


Agora que estamos com o nosso dataset no Colab podemos importá-lo para utilização.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

file_path = "/content/bank-full.csv"
df = pd.read_csv(file_path, sep=';')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


Para não se alongar muito, o foco dessa vez não é a transformação do dados para que possamos utilizaar no nosso modelo preditivo.

Para um passo a passo mais detalhado sobre a transformação das variáveis categóricas pode consultar no outro [notebook](https://colab.research.google.com/drive/1mzthkMQXYSnkyOWUsA0JqR14-ii3HkOf?usp=sharing) que eu elaborei.

Aqui vou avançar os passos e deixar registrado apenas o comandos.

In [ ]:
# Separar os dados entre feature matrix e target vector
X = df.drop('y', axis=1)
y = df['y']
X.shape,y.shape

((45211, 16), (45211,))

In [ ]:
# Convertendo as variáveis para inteiros 0,1.
y = pd.Series(np.where(y.values == 'yes',1,0), y.index)
y.value_counts()

0    39922
1     5289
dtype: int64

In [ ]:
#Label Enconder
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

#1. instanciando o enconder
label_encoder = LabelEncoder()

#crinado uma lista vazia para receber esses dados
mapping = []

#definindo a ordem dos rotulos para cada coluna categórica
educ_order = ['primary', 'secondary', 'tertiary', 'unknown']
month_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

#usando o cat.codes do Pandas para ordenar e por relevancia de cardinalidade
def order_labels(df, col, order):
  df[col] = df[col].astype('category')
  df[col] = df[col].cat.reorder_categories(order, ordered=True)
  df[col] = df[col].cat.codes.astype(int)

#usando dummies_var para a coluna 2-job
X = pd.concat([X, pd.get_dummies(X['job'])], axis=1).drop('job',axis=1)

#usando a função definida a cima para as colunas  11-mount e 4-education 
order_labels(X, 'education', educ_order)
order_labels(X, 'month', month_order)

#usando Label Enconding para as demais variáveis de baixa cardinalidade
for i, col in enumerate(X):
  if X[col].dtype == 'object':
    X[col] = label_encoder.fit_transform(np.array(X[col].astype(str)).reshape((-1,)))
    mapping.append(dict(zip(label_encoder.classes_,range(1, len(label_encoder.classes_)+1))))

X.head()

,age,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,admin.,blue-collar,entrepreneur,housemaid,management,retired,self-employed,services,student,technician,unemployed,unknown
0,58,1,2,0,2143,1,0,2,5,4,261,1,-1,0,3,0,0,0,0,1,0,0,0,0,0,0,0
1,44,2,1,0,29,1,0,2,5,4,151,1,-1,0,3,0,0,0,0,0,0,0,0,0,1,0,0
2,33,1,1,0,2,1,1,2,5,4,76,1,-1,0,3,0,0,1,0,0,0,0,0,0,0,0,0
3,47,1,3,0,1506,1,0,2,5,4,92,1,-1,0,3,0,1,0,0,0,0,0,0,0,0,0,0
4,33,2,3,0,1,0,0,2,5,4,198,1,-1,0,3,0,0,0,0,0,0,0,0,0,0,0,1


Pronto, aqui já alteramos as variáveis categóricas, agora iremos utilizar o método Ensemble, que consiste basicamente em combinar as previsões de diversos modelos, sendo que a previsão final é aquela que ocorre com maior frequência.

A ideia por trás do Ensemble é combinar conceitualmente diferentes modelos classificadores de machine learning e utilizar o comitê de votação ou a médias das probabilidades de previsões para predizer qual será o resultado final em função das diversas variáveis dos nossos dados.

Tal classificador pode ser útil para um conjunto de modelos de desempenho igualmente bom, a fim de equilibrar suas fraquezas individuais

## Ensemble

Você entenderá a importância do método de `Ensemble` ao entrar no [universo do Machine Learning](https://sigmoidal.ai/como-salvar-seu-modelo-de-machine-learning/) e ficar perdido com a quantidade de modelos diferentes que temos a disposição. Temos regressão linear, polinomial e logística, gradiente descendente, [XGBoost](https://sigmoidal.ai/xgboost-aprenda-algoritmo-de-machine-learning-em-python/), máquina de vetores de suporte, naive bayes, árvores de decisão, Random Forest, entre outros.

Observe abaixo o resultado do classificador de votação para os diferentes modelos. No caso, o VotingClassifier fez a combinação dos modelos.

A documentação original sobre a metodologia pode ser consultada neste [link](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html).


### Modelos Utilizados na avaliação

Como dito anteriorente o método `ensemble` nos permite utilizar diversos modelos em paralelo e extrair o resultado através de um comitê.



#### XGBoost Classifier

Criado em 1999 por TIANQI CHEN a técnica é utilizada em modelos preditivos com dados regulares.

Pertencente a categória de árvores de decisão, sendo capaz de combinar resultados de diversos classificadores "fracos" (baixa correlação) mediante um comitê de decisão.

O nome XGBoost vem de e**X**treme **G**radient **Boost**ing!

<p align=center><img src="http://sigmoidal.ai/wp-content/uploads/2019/08/boosted_stumps.gif" width="30%"></p>

O XGBoost trabalha a partir de árvores individuais criadas utilizando múltpiplos cores e os dados são organizados de forma a minimizar o tempo de consulta entre os ramos.


####SGD Classifier

Classificadores lineares (SVM, regressão logística, etc.) com treinamento SGD.

Este estimador implementa modelos lineares regularizados com aprendizagem de gradiente descendente estocástico (SGD): o gradiente da perda é estimado a cada amostra por vez e o modelo é atualizado ao longo do caminho com um cronograma de força decrescente (também conhecido como taxa de aprendizagem).

<center><img alt="SGDC" width="30%" src="https://miro.medium.com/max/710/1*Sa5kGcZIVNTLjrI8P-YsSQ.gif"></center>

[documentação oficial](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html?highlight=sgdclassifier#sklearn.linear_model.SGDClassifier)

#### Decision Tree Classifier

Decision Tree, ou Árvore de Decisão em português, é um algoritmo de aprendizado supervisionado para Machine Learning, e pode ser usado tanto para classificação quanto para regressão.

Uma Decision Tree é construída a partir de um processo de indução, que vai dividindo os dados em subconjuntos cada vez mais puros. Para selecionar os atributos que irão compor a árvore, são considerados o grau de entropia de um nó e a informação ganha a mais após um split.

<center><img alt="DecisionTree" width="50%" src="https://projetos.unisanta.br/computacao/gradu06/ShellSE/arvore.JPG"></center>

#### SVC - Support Vector Classifier

Um dos modelos de SVM - Support Vector Machines. É um modelo de algoritmo de aprendizado profundo (deep learing) que realiza aprendizado supervisionado para classificação ou regressão de grupos de dados.

### Executando previsões

Agora que sabemos um pouco sobre cada modelo vamos executá-los e utilizar o `Ensemble` para verificar os resultados.

In [ ]:
# importar os pacotes necessários
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [46]:
#np.random.seed(2)

# 1. escolher e importar um modelo
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

# 2. Instanciar e escolher os hyperparameters
model_xgbc = XGBClassifier()
model_sgd = SGDClassifier()
model_svc = SVC()
model_dt = DecisionTreeClassifier()
voting_clf = VotingClassifier(estimators=[('xgbc', model_xgbc), ('sgd', model_sgd),('svc', model_svc),('dt', model_dt)])

# 3. Separar os dados entre feature matrix e target vector 
# os dados já foram separados anteiormente

# 3.1 Dividir o dataset entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 3.2 Padronizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Fit do modelo (treinar)
for model in (model_xgbc, model_sgd, model_svc, model_dt, voting_clf):
    model.fit(X_train_scaled, y_train)

# 5. Fazer previsões em cima de novos dados
model = []
accuracy = []
for clf in (model_xgbc, model_sgd, model_svc, model_dt, voting_clf):
    y_pred = clf.predict(X_test_scaled)
    model.append(clf.__class__.__name__)
    accuracy.append(accuracy_score(y_test,y_pred))

# Verificar a acurácia
col = ['Acurácia']
ac = pd.DataFrame(data=accuracy, index = model, columns=col)
ac

,Acurácia
XGBClassifier,0.901088
SGDClassifier,0.887464
SVC,0.895161
DecisionTreeClassifier,0.875697
VotingClassifier,0.893745


### Métricas de avaliação

In [49]:
from sklearn import metrics
report = metrics.classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.90      0.99      0.94      9950
           1       0.71      0.19      0.30      1353

    accuracy                           0.89     11303
   macro avg       0.81      0.59      0.62     11303
weighted avg       0.88      0.89      0.87     11303



Do report anterior podemos verificar que o nosso modelo acertou 90% das previsões onde a resposta a campanha de marketing foi negativa e acertou 71% das vezes onde foi positiva.

Para saber mais sobre as métricas de classificação e avaliação de modelos pode acessar [meu post](https://mabittar.github.io/Metricas/) sobre o assunto.

#Conclusão

No nosso modelos em questão, a Acurácia nos mostra que 89% das vezes em que o modelo previu que sim ou não estava correto. 


Mesmo cometendo erros, o classificador geralmente consegue performar melhor do que os estimadores individualmente. De acordo com Aurélien Géron:,

<center>Mesmo que cada estimador seja um aprendiz fraco (o que significa que sua classificação é apenas um pouco melhor do que adivinhações aleatórias), o conjunto ainda pode ser um forte aprendiz (alcançando alta acurácia).</center>


Para o nosso exemplo, o modelo XGBoost é o que performa melhor entre os modelos individuais, mas ainda assim é uma performance próxima ao classificador de votação.

# Bonus

Podemos ainda utilizar ainda outro recurso do Scikit-Learn e verificar como os resultados se compararam

## Utilizaçã do KFold para Validação Cruzada

Ao invés de utilizarmos o `train,test,split` iremos utilizar o KFold que irá subdividir o dataset em diversas partes randomicas a fim de evitarmos um possível outlier que contamine nossa avaliação.


In [ ]:
X_Falso = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
resultado = []

for rep in range(10):

  kf = KFold(2, shuffle=True, random_state=rep)
  print("rep:", rep)
  # 2. Instanciar e escolher os hyperparameters
  model_sgd = SGDClassifier()
  model_svc = SVC()
  model_dt = DecisionTreeClassifier()
  voting_clf = VotingClassifier(estimators=[('sgd', model_sgd),('svc', model_svc),('dt', model_dt)])


  # 3. Divisão dos dados - para gerarmos diversas divisões diferentes inserimos o a função dentro de um for
  for linhas_treino, linhas_teste in kf.split(X):
    print("Treino:", linhas_treino.shape[0])
    print("Teste:", linhas_teste.shape[0])

    #é necessário selecionar manualmente as linhas para teste e treino
    X_train_kf,X_test_kf = X.iloc[linhas_treino], X.iloc[linhas_teste]
    y_train_kf, y_test_kf = y.iloc[linhas_treino], y.iloc[linhas_teste]
    # print(X_train_kf) #apenas para fins didático é possível imprimir as linhas

    # 3.2 Padronizar os dados
    scaler = StandardScaler()
    X_train_scaled_kf = scaler.fit_transform(X_train_kf)
    X_test_scaled_kf = scaler.transform(X_test_kf)

    #4. Instanciando o modelo
    model = SVC()
    model.fit(X_train_scaled_kf, y_train_kf)
    #5. Previsões
    y_pred_kf = model.predict(X_test_scaled_kf)
    #6. Armazendo resultados
    acc = np.mean(y_test_kf == y_pred_kf)
    resultado.append(acc)
    print("Acuracia:", acc)
    print()



rep: 0
Treino: 30140
Teste: 15071
Acuracia: 0.895229248225068

Treino: 30141
Teste: 15070
Acuracia: 0.8998672859986728

Treino: 30141
Teste: 15070
Acuracia: 0.8965494359654944

rep: 1
Treino: 30140
Teste: 15071
Acuracia: 0.9006701612368124

Treino: 30141
Teste: 15070
Acuracia: 0.8975447909754479

Treino: 30141
Teste: 15070
Acuracia: 0.8954877239548772

rep: 2
Treino: 30140
Teste: 15071
Acuracia: 0.9001393404551788

Treino: 30141
Teste: 15070
Acuracia: 0.8978102189781022

Treino: 30141
Teste: 15070
Acuracia: 0.8959522229595223

rep: 3
Treino: 30140
Teste: 15071
Acuracia: 0.8973525313516024

Treino: 30141
Teste: 15070
Acuracia: 0.9013271400132714

Treino: 30141
Teste: 15070
Acuracia: 0.8939615129396151

rep: 4
Treino: 30140
Teste: 15071
Acuracia: 0.9004047508459956

Treino: 30141
Teste: 15070
Acuracia: 0.8964167219641672

Treino: 30141
Teste: 15070
Acuracia: 0.8952222959522229

rep: 5
Treino: 30140
Teste: 15071
Acuracia: 0.8979497047309403

Treino: 30141
Teste: 15070
Acuracia: 0.89754479

Após realizarmos diversas iterações com o nosso dataset (número de reps) podemos obter a acurácia média pela fórmula:

In [ ]:
np.mean(resultado)

0.8976244635526032

## Comparação entre os resultados

In [47]:
print("Resultado inicial:")
print (ac)
print("\nResultado utilizando o KFold:\t", np.mean(resultado))


Resultado inicial:
                        Acurácia
XGBClassifier           0.901088
SGDClassifier           0.887464
SVC                     0.895161
DecisionTreeClassifier  0.875697
VotingClassifier        0.893745

Resultado utilizando o KFold:	 0.8938172195954411


Mesmo alterando de forma aleatória nossas amostras de teste e treinamento o modelo prditivo não se alterou.
